<a href="https://colab.research.google.com/github/zi-onion/ESAA/blob/main/05_24_K_means%2CCluster_Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[​과제 : 파이썬 머신러닝 완벽가이드 ch7.1-2 pg.409-427]

# CH7 군집화

## 01 K-평균 알고리즘 이해

K-평균은 군집화에 가장 일반적으로 사용되는 알고리즘으로, 군집 중심점(centroid)이라는 특정한 임의의 지점을 선택해 해당 중심에 가장 가까운 포인트들을 선택하는 군집화 기법이다.
선택된 포인트의 평균 지점으로 이동하고 이동된 중심점에서 다시 가까운 포인트를 선택, 다시 중심점을 평균 지점으로 이동하는 프로세스를 반복적으로 수행한다.

* K-평균의 장점
  - 일반적인 군집화에서 가장 많이 활용되는 알고리즘이다.
  - 알고리즘이 쉽고 간결하다.
* K-평균의 단점
  - 거리 기반 알고리즘으로 속성의 개수가 매우 많을 경우 군집화 정확도가 떨어진다.
  - 반복을 수행하는데,반복 횟수가 많을 경우 수행 시간이 매우 느려진다.
  - 몇 개의 군집(cluster)을 선택해야 할 지 가이드하기 어렵다.

### 사이킷런 KMeans 클래스 소개

초기 파라미터

> class sklearn. cluster. KMeans(n_clusters, init=k-meanst', n_init=10, max_iter=300, to1=0.0001, precompute_distances='auto', verbose=0, random_state-None, copy_x=True, n_jobs=1, algorithm='auto')

* 주요 파라미터
  - 가장 중요한 파라미터 n_clusters: 군집화할 개수, 즉 군집중심점의 개수를 의
미
  - init: 초기에 군집중심점의 좌표를 설정할 방식. 보통은 임의로 중심을 설정하지 않고 일반적으로 K-means++방식으로 최초 설정.
  - max_iter: 최대 반복 횟수. 이 횟수 이전에 모든 데이터의 중심점 이동이 없으면 종료

* 주요 속성 정보
  - labels: 각 데이터 포인트가 속한 군집 중심점 레이블
  - cluster_centers_: 각 군집중심점 좌표(Shape는[군집개수, 피처개수]). 군집 중심점 좌표가 어디인지 시각화할 수 있다.

### K-평균을 이용한 붓꽃데이터 세트 군집화

In [ ]:
from sklearn.preprocessing import scale
from sklearn.datasets import load_iris
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline

iris = load_iris()
# 더 편리한 데이터 핸들링을 위해 Data frame으로 변환
irisDF = pd.DataFrame(data=iris.data, columns=['sepal_length', 'sepal width', 'petal_length', 'petal_width'])
irisDF.head(3)

In [ ]:
## 3개 그룹으로 군집화
kmeans = KMeans(n_clusters=3, init='k-means++', max_iter=300,random_state=0)
kmeans. fit(irisDF)

In [ ]:
print(kmeans.labels_)

In [ ]:
irisDF['target'] = iris.target
irisDF[ 'cluster']=kmeans.labels_
iris_result = irisDF.groupby(['target','cluster'])['sepal_length'].count()
print(iris_result)

In [ ]:
## 속성 2개로 차원 축소
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
pca_transformed = pca.fit_transform(iris.data)

irisDF['pca_x'] = pca_transformed[:, 0]
irisDF['pca_y'] = pca_transformed[:, 1]
irisDF.head(3)

In [ ]:
# 군집값이 0, 1, 2인 경우마다 별도의 인덱스로 추출
marker0_ind = irisDF[irisDF['cluster']==0].index
marker1_ind = irisDF[irisDF['cluster']==1].index
marker2_ind = irisDF[irisDF['cluster']==2].index

# 군집값0, 1, 2에 해당하는 인덱스로 각 군집 레벨의 pca_x,pca_y값 추출. 0, 5, ^로 마커 표시
plt.scatter(x=irisDF.loc[marker0_ind, 'pca_x'], y=irisDF.loc[marker0_ind, 'pca_y'], marker='o')
plt.scatter(x=irisDF.loc[marker1_ind, 'pca_x'], y=irisDF.loc[marker1_ind, 'pca_y'], marker='s')
plt.scatter(x=irisDF.loc[marker2_ind, 'pca_x'], y=irisDF.loc[marker2_ind, 'pca_y'], marker='^')

plt.xlabel('PCA 1')
plt.ylabel('PCA 2')
plt. title('3 Clusters Visualization by 2 PCA Components')
plt.show()

### 군집화 알고리즘 테스트를 위한 데이터 생성
여러 개의 클래스에 해당하는 데이터 세트를 만드는데, 하나의 클래스에 여러 개의 군집이 분포될 수 있게 데이터를 생성
* make_blobs(): 개별 군집의 중심점과 표준편차 제어 기능이 추가되어 있음.
  - n_samples: 생성할 총데이터의 개수. 디폴트는 100개
  - n_features: 데이터의 피처 개수.시각화를 목표로 할 경우 2개로 설정해 보통 첫번째 피처는 x좌표,두번째 피처는 y좌표상에 표현.
  - centers: int 값으로 설정하면 군집의개수를, ndarray형태로 표현할 경우 개별 군집 중심점의 좌표를 의미.
  - cluster_std: 생성될 군집 데이터의 표준편차.float값 0.8과 같은 형태로 지정하면 군집내에서 데이터가 표준편차 0.8을 가진 값으로 만들어짐.
  [0.8,1.2. 0.6]의 표현은 군집별로 서로 다른 표준편차를 가진 데이터 세트를 만들때 사용.

* make_casifcation():노이즈를 포함한 데이터를 만드는데 유용함.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn. cluster import KMeans
from sklearn.datasets import make_blobs
%matplotlib inline

X, y = make_blobs (n_samples=200, n_features=2, centers=3, cluster_std=0.8, random_state=0)
print(X.shape, y.shape)

# y target 값의 분포를 확인
unique, counts = np.unique(y, return_counts=True)
print(unique, counts)

In [ ]:
import pandas as pd

clusterDF =pd.DataFrame(data=X, columns=['ftr1', 'ftr2'])
clusterDF['target'] = y
clusterDF.head(3)

In [ ]:
target_list = np.unique(y)
# 각 타깃별 산점도의 마커 값.
markers=['o', 's', '^', 'P', 'D', 'H', 'x']
# 3개의 군집 영역으로 구분한 데이터 세트를 생성했으므로 target_list는 [0, 1, 2]
# target=0, target=1,target=2로 scatterplot을 marker별로 생성.
for target in target_list:
  target_cluster = clusterDF[clusterDF['target']==target]
  plt.scatter(x=target_cluster['ftr1'], y=target_cluster['ftr2'], edgecolor='k', marker=markers[target] )
plt.show()

In [ ]:
# KMeans 객체를 이용해 X 데이터를 K-means 클러스터링 수행
kmeans = KMeans(n_clusters=3, init='k-means++', max_iter=200, random_state=0)
cluster_labels = kmeans.fit_predict(X)
clusterDF['kmeans_label'] = cluster_labels

# cluster_centers_ 는 개별 클러스터의 중심 위치 좌표 시각화를 위해 추출
centers = kmeans.cluster_centers_
unique_labels = np.unique(cluster_labels)
markers=['o', 's', '^','P', 'D', 'H','x']

# 군집된 label 유형별로 iteration 하면서 marker별로 scatterplot수행.
for label in unique_labels:
  label_cluster = clusterDF[clusterDF['kmeans_label']==label]
  center_x_y =centers[label]
  plt.scatter(x=label_cluster['ftr1'], y=label_cluster['ftr2'], edgecolor='k', marker=markers[label])

  # 군집별 중심 위치 좌표 시각화
  plt.scatter(x=center_x_y[0], y=center_x_y[1], s=200, color='white',
              alpha=0.9, edgecolor='k', marker=markers[label])
  plt.scatter(x=center_x_y[0], y=center_x_y[1], s=70, color='k', edgecolor='k',
              marker='$%d$' % label)
plt.show()

In [ ]:
print(clusterDF.groupby('target')['kmeans_label'].value_counts())

##02 군집 평가(Cluster Evaluation)

### 실루엣 분석의 개요
군집화 평가 방법인 실루엣 분석(silhouette analysis)은 각 군집 간의 거리가 얼마나 효율적으로 분리돼 있는지를 나타낸다. 군집화가 잘 될수록 개별 군집은 비슷한 정도의 여유공간을 가지고 떨어져 있다.

개별 데이터가 가지는 군집화 지표인 실루엣 계수(silhouet tecoeficient)를 기반으로 분석한다.
i번째 데이터 포인트의 실루엣 계수 값 s(i)은 다음과 같다.

$$ s(i) = (b(i) - a(i))/max(a(i),b(i))$$

실루엣 계수는 1에서 1사이의 값을 가지며, 1로 가까워질수록 근처의 군집과 더 멀리 떨어져 있다는 것이고, 0에 가까울수록 근처의 군집과 가까워진다는 것이다. -값은 아예 다른 군집에 데이터 포인트가 할당됐음을 뜻한다.


* 좋은 군집화가 되기 위한 조건
1. 전체 실루엣 계수의 평균값, 즉 사이킷런의 sihouette_score()값은 0 ~ 1사이의 값을 가지며,1에 가까울수록 좋다.
2. 전체 실루엣 계수의 평균값과 더불어 개별 군집의 평균값의 편차가 크지 않아야 한다.


### 붓꽃 데이터 세트를 이용한 군집 평가

In [ ]:
from sklearn.preprocessing import scale
from sklearn.datasets import load_iris
from sklearn.cluster import KMeans

# 실루엣 분석 평가 지표 값을 구하기 위한 API 추가
from sklearn.metrics import silhouette_samples, silhouette_score
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline

iris = load_iris()
feature_names = ['sepal_length', 'sepal width', 'petal_length', 'petal_width']
irisDF = pd.DataFrame(data=iris.data, columns=feature_names)
kmeans = KMeans(n_clusters=3, init='k-means++', max_iter=300, random_state=0).fit(irisDF)
irisDF['cluster'] = kmeans.labels_

# iris의 모든 개별 데이터에 실루엣 계수 값을 구함.
score_samples = silhouette_samples(iris.data, irisDF['cluster'])
print('silhouette_samples () return 값의 shape', score_samples.shape)

# irisDF에 실루엣 계수 칼럼 추가
irisDF['silhouette_coeff'] = score_samples

# 모든 데이터의 평균 실루엣 계수 값을 구함.
average_score = silhouette_score(iris.data, irisDF['cluster'])
print('붓꽃 데이터 세트 Silhouette Analysis Score:{0:.3f}'.format(average_score))
irisDF.head(3)

-> 평균 실루엣 계수 값이 0.553

In [ ]:
irisDF.groupby('cluster')['silhouette_coeff'].mean()

### 군집별 평균 실루엣 계수의 시각화를 통한 군집 개수 최적화 방법

개별 군집별로 적당히 분리된 거리를 유지하면서도 군집 내 데이터가 서로 뭉쳐 있는 경우에 K-평균의 적절한 군집화 개수가 설정됐다고 판단 가능

In [ ]:
def visualize_silhouette(cluster_lists, X_features):
    from sklearn.datasets import make_blobs
    from sklearn.cluster import KMeans
    from sklearn.metrics import silhouette_samples, silhouette_score

    import matplotlib.pyplot as plt
    import matplotlib.cm as cm
    import math

    #입력값으로 클러스터링 갯수들을 리스트로 받아서, 각 갯수별로 클러스터링을 적용하고 실루엣 개수를 구함
    n_cols = len(cluster_lists)

    #plt.subplots()으로 리스트에 기재된 클러스터링 수만큼의 sub figures를 가지는 axs 생성
    fig, axs = plt.subplots(figsize=(4*n_cols, 4), nrows=1, ncols=n_cols)

    #리스트에 기재된 클러스터링 갯수들을 차례로 iteration 수행하면서 실루엣 개수 시각화
    for ind, n_cluster in enumerate(cluster_lists):

        #KMeans 클러스터링 수행하고, 실루엣 스코어와 개별 데이터의 실루엣 값 계산.
        clusterer = KMeans(n_clusters = n_cluster, max_iter=500, random_state=0)
        cluster_labels = clusterer.fit_predict(X_features)

        sil_avg = silhouette_score(X_features, cluster_labels)
        sil_values = silhouette_samples(X_features, cluster_labels)

        y_lower = 10
        axs[ind].set_title('Number of Cluster : '+ str(n_cluster)+'\n' \
                          'Silhouette Score :' + str(round(sil_avg,3)) )
        axs[ind].set_xlabel("The silhouette coefficient values")
        axs[ind].set_ylabel("Cluster label")
        axs[ind].set_xlim([-0.1, 1])
        axs[ind].set_ylim([0, len(X_features) + (n_cluster + 1) * 10])
        axs[ind].set_yticks([])  # Clear the yaxis labels / ticks
        axs[ind].set_xticks([0, 0.2, 0.4, 0.6, 0.8, 1])

        #클러스터링 갯수별로 fill_betweenx( )형태의 막대 그래프 표현.
        for i in range(n_cluster):
            ith_cluster_sil_values = sil_values[cluster_labels==i]
            ith_cluster_sil_values.sort()

            size_cluster_i = ith_cluster_sil_values.shape[0]
            y_upper = y_lower + size_cluster_i

            color = cm.nipy_spectral(float(i) / n_cluster)
            axs[ind].fill_betweenx(np.arange(y_lower, y_upper), 0, ith_cluster_sil_values, \
                                facecolor=color, edgecolor=color, alpha=0.7)
            axs[ind].text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))
            y_lower = y_upper + 10

        axs[ind].axvline(x=sil_avg, color="red", linestyle="--")


In [ ]:
#make_blobs 군집화를 위한 4개의 군집 중심의 500개 2차원데이터 생성
X,y = make_blobs(n_samples = 500, n_features = 2, centers = 4, cluster_std = 1, center_box=(-10.0, 10.0), shuffle = True, random_state = 0)
visualize_silhouette([2,3,4,5], X)

#군집 개수를 2개, 3개, 4개, 5개 일때의 군집별 실루엣 계수 평균값을 시각화
visualize_silhouette([ 2, 3, 4, 5], X)

In [ ]:
from sklearn.datasets import load_iris
iris = load_iris()
visualize_silhouette([2,3,4,5], iris.data)

K-평균으로 군집화할 경우에는 군집 개수를 2개로 하는 것이 가장 좋아보임.